# LeafGan Project

# setup

In [1]:
import numpy as np
import pandas as pd
import os
from glob import glob
import torch
from PIL import Image
from torch import nn
import numpy as np
import cv2
import torchvision 
from torchvision import transforms, models
import matplotlib.pyplot as plt
from tqdm import tqdm
from zipfile import ZipFile
from sklearn.model_selection import StratifiedKFold
import plotly.graph_objects as go
from sklearn.preprocessing import MultiLabelBinarizer
from torch.utils.data import DataLoader, Dataset
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
from sklearn.metrics import classification_report

In [2]:
!pip uninstall opencv-python-headless==4.5.5.62
!pip install opencv-python-headless==4.5.2.52

     |████████████████████████████████| 38.2 MB 1.2 MB/s 


In [3]:
!pip install pytorch-ignite
from ignite.metrics import ClassificationReport

     |████████████████████████████████| 251 kB 4.2 MB/s 


In [4]:
!pip install torchmetrics
import torchmetrics

     |████████████████████████████████| 397 kB 4.1 MB/s 


In [5]:
!pip install -U git+https://github.com/albu/albumentations > /dev/null 
import albumentations as A
from albumentations.pytorch import ToTensorV2

  Running command git clone -q https://github.com/albu/albumentations /tmp/pip-req-build-a0r2zkio


**login to weights and biases**

**link to wandb guide** - https://docs.wandb.ai/quickstart

In [6]:
!pip install wandb
import wandb

     |████████████████████████████████| 1.7 MB 4.2 MB/s 
     |████████████████████████████████| 144 kB 71.6 MB/s 
     |████████████████████████████████| 180 kB 89.4 MB/s 
     |████████████████████████████████| 63 kB 2.6 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=42aaf8c9e2b222107daf9b9eb884183a1c3db76ce4424abfd93b74dd1fc45ef0
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [7]:

#!wandb login 52b3b279a9d634741e39bd8541fece124dfb8b88

**connect to google Drive**

In [8]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


# Load dataset 
download the Plant Pathology 2021 dataset with kaggle API to google drive

to download API Credentials follow the instructions in the following link - https://www.analyticsvidhya.com/blog/2021/06/how-to-load-kaggle-datasets-directly-into-google-colab/  

In [ ]:
! pip install kaggle
! mkdir /.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
%cd /gdrive/MyDrive

/gdrive/MyDrive


make directories

In [ ]:
!mkdir LeafGANDataSet
!mkdir plant-pathology-dataset

**download the plant pathology 2021 900x900 images from kaggle dataset**

In [ ]:
! kaggle datasets download jirkaborovec/plant-pathology-2021-fgvc8-960px -p /gdrive/MyDrive/LeafGANDataSet/plant-pathology-dataset

100% 1.21G/1.21G [00:42<00:00, 24.2MB/s]
100% 1.21G/1.21G [00:43<00:00, 30.3MB/s]


enter to the right path

In [32]:
%cd /gdrive/MyDrive/LeafGANDataSet/plant-pathology-dataset/

/gdrive/.shortcut-targets-by-id/19BTxmd5iqi13fW526v_d0CgS_shMILpC/LeafGANDataSet/plant-pathology-dataset


unzip files

In [ ]:
!unzip /gdrive/MyDrive/LeafGANDataSet/plant-pathology-dataset/plant-pathology-2021-fgvc8-960px.zip

In [ ]:
%cd /gdrive/MyDrive/LeafGANDataSet

/gdrive/.shortcut-targets-by-id/19BTxmd5iqi13fW526v_d0CgS_shMILpC/LeafGANDataSet


# Train resnet model 
train resnet152 to classify apple disease from the Plant Pathology 2021 dataset
and analize classes matrics performance.
this will help understand wich of the class need more images generated by the LeafGan

**read train csv file**

In [7]:
dataset_path = "/gdrive/MyDrive/LeafGANDataSet/"
df = pd.read_csv(dataset_path + 'train.csv', index_col="image")

**keep only single labeled samples**

In [8]:
df1 = df[df['labels']== 'scab']
df2 = df[df['labels']=='healthy']
df3 = df[df['labels']=='frog_eye_leaf_spot']
df4 = df[df['labels']=='rust']
df5 = df[df['labels']=='complex']
df6 = df[df['labels']=='powdery_mildew']
df = pd.concat([df1, df2, df3, df4, df5, df6])

In [19]:
df.head()

,labels
image,
80070f7fb5e2ccaa.jpg,scab
80077517781fb94f.jpg,scab
80230a9a3f7a9f6b.jpg,scab
80261f473eafb92c.jpg,scab
802962dc3ecdbb8d.jpg,scab


In [20]:
source = df['labels'].value_counts()
fig = go.Figure(data=[go.Pie(labels=source.index,values=source.values)])
fig.update_layout(title='Label distribution all dataset')
fig.show()

split the data - 80% train, 20% test with stratified 5 Folds to preserve labels distribution

In [10]:
sfk = StratifiedKFold(5)
for train_idx, valid_idx in sfk.split(df.index, df['labels']):
    df_train = df.iloc[train_idx]
    df_valid = df.iloc[valid_idx]
    break
print(f"train size: {len(df_train)}")
print(f"valid size: {len(df_valid)}")

train size: 13821
valid size: 3456


In [22]:
source = df_train['labels'].value_counts()
fig = go.Figure(data=[go.Pie(labels=source.index,values=source.values)])
fig.update_layout(title='Label distribution train')
fig.show()

In [23]:
source = df_valid['labels'].value_counts()
fig = go.Figure(data=[go.Pie(labels=source.index,values=source.values)])
fig.update_layout(title='Label distribution validation')
fig.show()

convert labels format to one-hot encoding

In [ ]:
classes = ['complex','frog_eye_leaf_spot','powdery_mildew','rust','scab','healthy']

original_labels = df_train['labels'].values.copy()
df_train['labels'] = [x.split(' ') for x in df_train['labels']]
labels = MultiLabelBinarizer(classes=classes).fit_transform(df_train['labels'].values)
df_train = pd.DataFrame(columns=classes, data=labels, index=df_train.index)

original_labels = df_valid['labels'].values.copy()
df_valid['labels'] = [x.split(' ') for x in df_valid['labels']]
labels = MultiLabelBinarizer(classes=classes).fit_transform(df_valid['labels'].values)
df_valid = pd.DataFrame(columns=classes, data=labels, index=df_valid.index)

transform for data augmentation

In [12]:
def get_transform(phase: str):
    if phase == 'train':
        return A.Compose([
            A.RandomResizedCrop(height=224, width=224),
            A.HorizontalFlip(p=0.5),
            A.ShiftScaleRotate(p=0.5),
            A.OneOf([
            A.RandomBrightnessContrast(p=0.5),A.RandomGamma(p=0.5)], p=0.5),
            A.OneOf([
            A.Blur(p=0.4),
            A.GaussianBlur(p=0.4),
            A.MotionBlur(p=0.4)], p=0.4),
            A.OneOf([
            A.GaussNoise(p=0.4),
            A.ISONoise(p=0.4),
            A.GridDropout(ratio=0.5, p=0.2),
            A.CoarseDropout(max_holes=16, min_holes=8, max_height=16, max_width=16, min_height=8, min_width=8, p=0.2)], p=0.2),
            A.Normalize(),
            ToTensorV2(),
        ])
    else:
        return A.Compose([
            A.Resize(height=224, width=224),
            A.Normalize(),
            ToTensorV2(),
        ])

Dataset Class

In [13]:
class PlantDataset(Dataset):
    def __init__(self, df, train_image_path,  test_image_path, transform=None, kind='train'):
        self.image_id = df.index
        self.labels = df.iloc[:, :].values
        self.transform = transform
        self.kind = kind
        self.train_image_path = train_image_path
        self.test_image_path = test_image_path

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image_id = self.image_id[idx]
        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        index = torch.argmax(label)
        if self.kind == 'train':
            image_path = self.train_image_path + '/' + image_id
        elif self.kind == 'test':
            image_path = self.test_image_path + '/' + image_id
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            image = self.transform(image=image)["image"]
        return image, label, index

create loaders

In [14]:
train_image_path = "/gdrive/MyDrive/LeafGANDataSet/plant-pathology-dataset/train_images"
test_image_path = "/gdrive/MyDrive/LeafGANDataSet/plant-pathology-dataset/test_images"

train_dataset = PlantDataset(df_train, train_image_path, test_image_path, get_transform('train'))
valid_dataset = PlantDataset(df_valid, train_image_path, test_image_path, get_transform('valid'))

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4, pin_memory=True, drop_last=True)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False, num_workers=4)

## training pytorch resnet152 network

In [15]:
def to_numpy(tensor):
    """Auxiliary function to convert tensors into numpy arrays
    """
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x)
    return e_x / e_x.sum(axis=0) # only difference

def Sigmoid(x):
    return 1/(1 + np.exp(-x))

In [16]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [32]:
def train_epoch(dataloader, model, criterion, optimizer, epoch):
    m = nn.Sigmoid()
    accuracy = AverageMeter()
    acc_cls1 = AverageMeter()
    acc_cls2 = AverageMeter()
    acc_cls3 = AverageMeter()
    acc_cls4 = AverageMeter()
    acc_cls5 = AverageMeter()
    acc_cls6 = AverageMeter()
    acc_list = [acc_cls1,
                acc_cls2,
                acc_cls3,
                acc_cls4,
                acc_cls5,
                acc_cls6]
    losses = AverageMeter()
    model.train()
    all_pred = []
    all_target = []
    for i, (img, target, target_num) in tqdm(enumerate(dataloader), total=len(dataloader), desc="Train"):
        img = img.float()    

        img = img.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        output = model(img)
        
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        pred = m(output)
        
        acc = torchmetrics.functional.accuracy(pred, target.int())
        accuracy_per_class = torchmetrics.functional.accuracy(pred, target.int(), average = 'none', num_classes = 6)
        target_num_unique, target_num_count = torch.unique(target_num.int(), return_counts = True)

        for i,j in zip(target_num_unique, target_num_count):
            acc_list[i.int()].update(accuracy_per_class[i.int()], j.int())

        losses.update(loss.item(), img.size(0))
        accuracy.update(acc, img.size(0))

        target = torch.argmax(target, dim=1).tolist()
        pred = torch.argmax(pred, dim=1).tolist()
        
        all_target += target
        all_pred += pred
        
        torch.cuda.empty_cache()
    
    cr = classification_report(all_target, all_pred, output_dict=True)

    wandb.log({"train loss": losses.avg, "train acc": accuracy.avg*100,
               "train precison": cr['macro avg']['precision']*100, "train recall": cr['macro avg']['recall']*100,
               "train f1": cr['macro avg']['f1-score']*100, "train precison cls1": cr['0']['precision']*100, "train recall cls1": cr['0']['recall']*100,
               "train f1 cls1": cr['0']['f1-score']*100, "train precison cls2": cr['1']['precision']*100, "train recall cls2": cr['1']['recall']*100,
               "train f1 cls2": cr['1']['f1-score']*100, "train precison cls3": cr['2']['precision']*100, "train recall cls3": cr['2']['recall']*100,
               "train f1 cls3": cr['2']['f1-score']*100, "train precison cls4": cr['3']['precision']*100, "train recall cls4": cr['3']['recall']*100,
               "train f1 cls4": cr['3']['f1-score']*100, "train precison cls5": cr['4']['precision']*100, "train recall cls5": cr['4']['recall']*100,
               "train f1 cls5": cr['4']['f1-score']*100, "train precison cls6": cr['5']['precision']*100, "train recall cls6": cr['5']['recall']*100,
               "train f1 cls6": cr['5']['f1-score']*100,
               "train acc cls1": acc_cls1.avg*100,
              "train acc cls2": acc_cls2.avg*100, "train acc cls3": acc_cls3.avg*100,
              "train acc cls4": acc_cls4.avg*100, "train acc cls5": acc_cls5.avg*100,
              "train acc cls6": acc_cls6.avg*100}, step = epoch)

In [40]:
def validate_epoch(dataloader, model, criterion, optimizer, epoch):
    m = nn.Sigmoid()
    acc = AverageMeter()
    acc_cls1 = AverageMeter()
    acc_cls2 = AverageMeter()
    acc_cls3 = AverageMeter()
    acc_cls4 = AverageMeter()
    acc_cls5 = AverageMeter()
    acc_cls6 = AverageMeter()
    acc_list = [acc_cls1,
                acc_cls2,
                acc_cls3,
                acc_cls4,
                acc_cls5, acc_cls6]
    losses = AverageMeter()
    all_pred = []
    all_target = []
    model.eval()
    loss_accumlated = 0
    for i, (img, target, target_num) in tqdm(enumerate(dataloader), total=len(dataloader), desc="Val"):
        
        img = img.float()
        img = img.to(device)
        target = target.to(device)
        target_num = target_num.to(device)
        
        optimizer.zero_grad() 
        
        output = model(img) 
        
        loss = criterion(output,target)
        losses.update(loss.item(), img.size(0))
        loss_accumlated += loss.item()
                
        pred = m(output)
                
        accuracy = torchmetrics.functional.accuracy(pred,  target.int())
        accuracy_per_class = torchmetrics.functional.accuracy(pred, target.int(), average = 'none', num_classes = 6)
        acc.update(accuracy, img.size(0))
        
        target_num_unique, target_num_count = torch.unique(target_num.int(), return_counts = True)

        for i,j in zip(target_num_unique, target_num_count):
            acc_list[i.int()].update(accuracy_per_class[i.int()], j.int())
        

        target = torch.argmax(target, dim=1).tolist()
        pred = torch.argmax(pred, dim=1).tolist()
        
        all_target += target
        all_pred += pred

        torch.cuda.empty_cache()

    cr = classification_report(all_target, all_pred, output_dict=True)
    wandb.log({"val loss": losses.avg, "val acc": acc.avg*100,
               "val precison": cr['macro avg']['precision']*100, "val recall": cr['macro avg']['recall']*100,
               "val f1": cr['macro avg']['f1-score']*100, "val precison cls1": cr['0']['precision']*100, "val recall cls1": cr['0']['recall']*100,
               "val f1 cls1": cr['0']['f1-score']*100, "val precison cls2": cr['1']['precision']*100, "val recall cls2": cr['1']['recall']*100,
               "val f1 cls2": cr['1']['f1-score']*100, "val precison cls3": cr['2']['precision']*100, "val recall cls3": cr['2']['recall']*100,
               "val f1 cls3": cr['2']['f1-score']*100, "val precison cls4": cr['3']['precision']*100, "val recall cls4": cr['3']['recall']*100,
               "val f1 cls4": cr['3']['f1-score']*100, "val precison cls5": cr['4']['precision']*100, "val recall cls5": cr['4']['recall']*100,
               "val f1 cls5": cr['4']['f1-score']*100, "val precison cls6": cr['5']['precision']*100, "val recall cls6": cr['5']['recall']*100,
               "val f1 cls6": cr['5']['f1-score']*100,
               "val acc cls1": acc_cls1.avg*100,
              "val acc cls2": acc_cls2.avg*100, "val acc cls3": acc_cls3.avg*100,
              "val acc cls4": acc_cls4.avg*100, "val acc cls5": acc_cls5.avg*100,
              "val acc cls6": acc_cls6.avg*100}, step = epoch)
    print(classification_report(all_target, all_pred))
    return cr['macro avg']['f1-score']*100

In [19]:
def train(loader_train, loader_val, EPOCHS, model, opt, criterion, model_path):
    best_acc = 0
    for e in range(EPOCHS):
        print("------------------------epoch " + str(e) + "-----------------------------")
        train_epoch(loader_train, model, criterion, opt, e)
        current_acc = validate_epoch(loader_val, model, criterion, opt, e)
        if best_acc < current_acc:
            best_acc = current_acc
            model_name = "resnet152_epoch" + str(e)
            torch.save(model, model_path + model_name)

In [41]:
model = models.resnet152(pretrained = True)
in_features = model.fc.in_features
model.fc = torch.nn.Linear(in_features, 6)
model = model.cuda()
criterion = nn.BCEWithLogitsLoss()
opt = torch.optim.AdamW(model.parameters(), lr=0.0001)

here you need to fill your own project and entity names from wandb

In [42]:
project_name = "Plant_Pathology_Classification"
entity_name = "seminar"
wab = wandb.init(project=project_name, entity=entity_name)

train acc,▁
train acc cls1,▁
train acc cls2,▁
train acc cls3,▁
train acc cls4,▁
train acc cls5,▁
train acc cls6,▁
train f1,▁
train f1 cls1,▁
train f1 cls2,▁
train f1 cls3,▁


In [43]:
model_path = "/gdrive/MyDrive/LeafGANDataSet/plant-pathology-dataset/models/"
train(train_loader, valid_loader, 30, model, opt, criterion, model_path)
wab.finish()

------------------------epoch 0-----------------------------


Val: 100%|██████████| 216/216 [00:33<00:00,  6.52it/s]


              precision    recall  f1-score   support

           0       0.94      0.58      0.72       320
           1       0.90      0.94      0.92       637
           2       0.88      0.96      0.92       237
           3       0.87      0.97      0.91       372
           4       0.92      0.94      0.93       966
           5       0.94      0.96      0.95       924

    accuracy                           0.91      3456
   macro avg       0.91      0.89      0.89      3456
weighted avg       0.92      0.91      0.91      3456

------------------------epoch 1-----------------------------


Val: 100%|██████████| 216/216 [00:33<00:00,  6.52it/s]


              precision    recall  f1-score   support

           0       0.90      0.62      0.74       320
           1       0.90      0.96      0.93       637
           2       0.85      0.96      0.90       237
           3       0.88      0.96      0.92       372
           4       0.96      0.90      0.93       966
           5       0.91      0.97      0.94       924

    accuracy                           0.91      3456
   macro avg       0.90      0.89      0.89      3456
weighted avg       0.92      0.91      0.91      3456

------------------------epoch 2-----------------------------


...

Val: 100%|██████████| 216/216 [00:33<00:00,  6.45it/s]


              precision    recall  f1-score   support

           0       0.95      0.71      0.81       320
           1       0.91      0.97      0.94       637
           2       0.90      0.98      0.94       237
           3       0.92      0.97      0.95       372
           4       0.97      0.94      0.95       966
           5       0.95      0.98      0.97       924

    accuracy                           0.94      3456
   macro avg       0.93      0.93      0.93      3456
weighted avg       0.94      0.94      0.94      3456

------------------------epoch 29-----------------------------


Val: 100%|██████████| 216/216 [00:33<00:00,  6.42it/s]


              precision    recall  f1-score   support

           0       0.89      0.77      0.82       320
           1       0.96      0.94      0.95       637
           2       0.92      0.98      0.95       237
           3       0.91      0.98      0.94       372
           4       0.96      0.96      0.96       966
           5       0.97      0.98      0.97       924

    accuracy                           0.95      3456
   macro avg       0.93      0.93      0.93      3456
weighted avg       0.95      0.95      0.95      3456



train acc,▁▄▅▅▆▆▆▆▆▇▆▇▇▇▇▇▇▇▇▇▇██▇██████
train acc cls1,▁▃▃▄▄▅▅▆▅▆▅▆▅▆▆▆▇▆▆▇▇▇▇▇▇▇██▇▇
train acc cls2,▁▄▅▅▆▅▆▆▆▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇█▇█▇█▇
train acc cls3,▁▅▅▆▆▆▆▇▇▇▆▇▇▇▇▇▇▇▇▇████▇███▇█
train acc cls4,▁▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇█▇█▇█▇████
train acc cls5,▁▄▅▅▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇█▇▇▇▇█████
train acc cls6,▁▄▅▆▆▆▆▇▆▇▇▇▇▇▇▇█▇██▇██▇██████
train f1,▁▄▅▅▆▆▆▆▆▇▆▇▇▇▇▇▇▇▇▇███▇█▇████
train f1 cls1,▁▄▄▅▅▆▅▆▆▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇█▇██▇▇
train f1 cls2,▁▄▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇██▇█▇█▇██
train f1 cls3,▁▄▅▅▆▆▆▆▇▇▆▇▇▇▇▇▇▇█▇███▇▇▇████


# LFLSeg Dataset
The key idea of the LFLSeg is the introduction of the partial leaf class. Partial leaf images are created from full leaf images. From one single full leaf image, we crop 9 patches to create 9 images for partial leaf class.

Dataset will have 3 classes:

*   full leaf: image that contains a full leaf
*   partial leaf: image that contains a part of a full leaf
*   non-leaf: image that does not contain any part of a leaf

##Non Leaf Class Data
Using downlaoder from git - ImageNet-Datasets-Downloader, to downlaod random images from ImageNet  
number of classes - 100 and 150 images per class

output dir - /content/drive/MyDrive/LeafGANDataSet/non-leaf

In [ ]:
!git clone https://github.com/mf1024/ImageNet-Datasets-Downloader.git

In [ ]:
!python /content/ImageNet-Datasets-Downloader/downloader.py \
    -data_root /content/drive/MyDrive/LeafGANDataSet/non-leaf \
    -number_of_classes 100 \
    -images_per_class 150

extracting all images to single folder

In [ ]:
for dir in glob('/gdrive/MyDrive/LeafGANDataSet/non-leaf/imagenet_images/*'):
  for photo in glob(dir + '/*'):
    img_id = photo.split('/')[-1].split('.')[0]
    img_id = img_id + photo.split('/')[-2] + '.jpg'
    os.rename(photo, '/gdrive/MyDrive/LeafGANDataSet/non-leaf1/' + img_id)

##Full Leaf Class Data
for the full leaf class we used Plant Pathology 2021 - FGVC8  resized to (520,520) dataset.
steps - 

*   download resized (520,520) images zip file and upload to drive
*   filter only the following classes ⟹ [healthy, frog_eye_leaf_spot,rust,complex, powdery_mildew]

*   Extracting the images from the zip with this zipfile package




In [ ]:
ZipFile("/content/drive/MyDrive/LeafGANDataSet/file_name.zip").extractall("/content/drive/MyDrive/LeafGANDataSet/dest_file_to_extract")

##Partial Leaf Class Data


*   Working on kaggle notebook to accsess the original Dataset
*   Randomly Choosing 1/9 from the images, resize the images to (900, 900) and saving them
*   Cropping the resized images (900,900) to 9 patches (300,300) using Image_slicer package



In [ ]:
!pip install image_slicer
import image_slicer

In [ ]:
def generate_partial_leaf(folder):
    for filename in tqdm(os.listdir(folder)):
        id =  filename.split('.jpg')[:-1]
        if id == []:
            print("not an image")
        else:
            id =  filename.split('.jpg')[:-1][0]
            tiles = image_slicer.slice(folder+"/"+filename, 9, save=False)
            image_slicer.save_tiles(tiles, directory='saving/path/folder',\
                            prefix=str(id))

In [ ]:
train_df_path = "path/to/train.csv"
df = pd.read_csv(train_df_path)
df1 = df[df['labels']== 'scab']
df2 = df[df['labels']=='healthy']
df3 = df[df['labels']=='frog_eye_leaf_spot']
df4 = df[df['labels']=='rust']
df5 = df[df['labels']=='complex']
df6 = df[df['labels']=='powdery_mildew']
df_train = pd.concat([df1,df2,df3,df4,df5,df6])
df_train = df_train.reset_index(drop=True)
df_train.index = df_train.index + 1
size = int(len(df_train)/9)
df_to_patch = df_train.sample(size)
df_to_pathc_index = df_to_patch.index

In [ ]:
for i in range(size):
    image_path = df_to_patch['image'].iloc[i]
    id =  image_path.split('.jpg')[:-1][0]
    im = Image.open(train_image_path+image_path)
    im = im.resize((900,900))
    im.save(image_path)

In [ ]:
df_to_patch['labels'].value_counts()

In [ ]:
for i in range(size):
    image_path = df_to_patch['image'].iloc[i]
    id =  image_path.split('.jpg')[:-1][0]
    tiles = image_slicer.slice('/content/'+image_path, 9, save=False)
    image_slicer.save_tiles(tiles, directory='/content/drive/MyDrive/image_sz_5',\
                            prefix=str(id))

In [ ]:
generate_partial_leaf("resized_images_folder_path")

#Split Data into train and test

**LeafGAN training**
We created folders for each disease: 
*   Frog Eye Leaf Spot
*   Scab
*   Powdery Mildew
*   Rust
*   Complex

for each disease we split the data into 80% train and 20% test.

In every folder we made the following folders:
*   TrainA - 80% Healthy leaf images
*   TestA - 20% Healthy leaf images
*   TrainB - 80% Disease leaf images 
*   TestB - 20% Disease leaf images

to fit the correct dir structure the leafGan requires

**LFLSeg training**

we created two text files (train_data.txt and test_data.txt) as specifies in the Git LFLSeg section - 

/path/to/full_leaf/full_leaf_1.JPG, 0

... ... ...

/path/to/partial_leaf/partial_leaf_1.JPG, 1

... ... ...

/path/to/non_leaf/non_leaf_1.png, 2


In [ ]:
train_df_path = "path/to/train.csv"
df = pd.read_csv(train_df_path)

In [ ]:
df['labels'].value_counts()

scab                               4826
healthy                            4624
frog_eye_leaf_spot                 3181
rust                               1860
complex                            1602
powdery_mildew                     1184
scab frog_eye_leaf_spot             686
scab frog_eye_leaf_spot complex     200
frog_eye_leaf_spot complex          165
rust frog_eye_leaf_spot             120
rust complex                         97
powdery_mildew complex               87
Name: labels, dtype: int64

In [ ]:
len(df)

18632

**LeafGAN**

In [ ]:
import shutil
def generate_data(df_healthy, df, dest):
  train_size = int(len(df_healthy)*0.8)
  test_size = int(len(df_healthy)*0.2)
  train_size_d = int(len(df)*0.8)
  test_size_d = int(len(df)*0.2)
  for i in tqdm(range(train_size)): #loop for 80% TrainA
    source = "/content/drive/MyDrive/LeafGANDataSet/full leaf/" +df_healthy['image'].iloc[i]
    destination = "/content/drive/MyDrive/LeafGANDataSet/"+dest+"/trainA"
    shutil.copy(source, destination)
 
  for i in tqdm(range(train_size_d)): #loop for 80% TrainB
    source = "/content/drive/MyDrive/LeafGANDataSet/full leaf/" + df['image'].iloc[i]
    destination = "/content/drive/MyDrive/LeafGANDataSet/"+dest+"/trainB"
    shutil.copy(source, destination)

  for i in tqdm(range(test_size)): #loop for 20% TestA
    source = "/content/drive/MyDrive/LeafGANDataSet/full leaf/" + df_healthy['image'].iloc[i + train_size]
    destination = "/content/drive/MyDrive/LeafGANDataSet/"+dest+"/testA"
    shutil.copy(source, destination)
  
  for i in tqdm(range(test_size_d)): #loop for 20% TestB
    source = "/content/drive/MyDrive/LeafGANDataSet/full leaf/" + df['image'].iloc[i + train_size_d]
    destination = "/content/drive/MyDrive/LeafGANDataSet/"+dest+"/testB"
    shutil.copy(source, destination)

In [ ]:
df1 = df[df['labels']== 'scab']
df2 = df[df['labels']=='healthy']
df3 = df[df['labels']=='frog_eye_leaf_spot']
df4 = df[df['labels']=='rust']
df5 = df[df['labels']=='complex']
df6 = df[df['labels']=='powdery_mildew']

In [ ]:
generate_data(df2, df1, "Healthy2Scab")
generate_data(df2, df5, "Healthy2Complex")
generate_data(df2, df6, "Healthy2PowderyMildew")
generate_data(df2, df3, "Healthy2frog_eye_leaf_spot")
generate_data(df2, df4, "Healthy2rust")

**LFLSeg**

In [ ]:
count = 0
train_file = open("/gdrive/MyDrive/LeafGANDataSet/train_data.txt", 'w')
test_file = open("/gdrive/MyDrive/LeafGANDataSet/test_data.txt", 'w')

for dir in glob('/gdrive/MyDrive/LeafGANDataSet/Data/*'):
  path, dirs, files = next(os.walk(dir))
  amount = len(files)
  train_imgs = len(files)*0.8
  count_train = 0
  if dir.split('/')[-1] == 'full leaf':
    count = 0
  elif dir.split('/')[-1] == 'partial leaf':
    count = 1
  else:
    count = 2
  for photo in glob(dir + '/*'):
    if count_train <= train_imgs:
      train_file.write(photo + ", "+ str(count) + "\n")
    else:
      test_file.write(photo + ", "+ str(count) + "\n")
    count_train += 1

train_file.close()
test_file.close()

# Clone git into drive project folder

In [ ]:
%cd /gdrive/MyDrive/LeafGANDataSet

In [ ]:
!git clone https://github.com/IyatomiLab/LeafGAN.git

# Training the LFLSeg Module

In [ ]:
%cd /gdrive/MyDrive/LeafGANDataSet/LeafGAN/LFLSeg

**train**

In [ ]:
!python train_LFLSeg.py --train /gdrive/MyDrive/LeafGANDataSet/train_data.txt --test /gdrive/MyDrive/LeafGANDataSet/test_data.txt

export CUDA_VISIBLE_DEVICES=0,1
Train data: /gdrive/MyDrive/LeafGANDataSet/train_data.txt
Test data: /gdrive/MyDrive/LeafGANDataSet/test_data.txt
Save trained models to: ./trained_models/
/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:853: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "
/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:288: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataL

**infer**

In [ ]:
for img in glob('/gdrive/MyDrive/LeafGANDataSet/Healthy2Rust/testB/*'):
  PATH = img
  !python infer_LFLSeg.py  --input $PATH --threshold 0.45 --segment

# Training LeafGAN

**prepare masks for training LeafGAN**

In [ ]:
!python /gdrive/MyDrive/LeafGANDataSet/LeafGAN/prepare_mask.py --source /gdrive/MyDrive/LeafGANDataSet/Healthy2Rust --pretrain_path /gdrive/MyDrive/LeafGANDataSet/LeafGAN/LFLSeg/trained_models/trained_resnet101_LFLSeg_v1_90.pth --image_size 224

##### trainA #####
  0% 0/3699 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:1025: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
100% 3699/3699 [10:51<00:00,  5.68it/s]
##### trainB #####
100% 1488/1488 [04:17<00:00,  5.77it/s]
##### testB #####
100% 372/372 [01:05<00:00,  5.67it/s]
##### testA #####
100% 924/924 [02:45<00:00,  5.57it/s]
done


**train**

In [9]:
%cd /gdrive/MyDrive/LeafGANDataSet/LeafGAN

/gdrive/.shortcut-targets-by-id/19BTxmd5iqi13fW526v_d0CgS_shMILpC/LeafGANDataSet/LeafGAN


In [10]:
!pip install dominate

In [ ]:
!python train.py --dataroot /gdrive/MyDrive/LeafGANDataSet/Healthy2Rust --name Healthy2Rust_leafGAN --model leaf_gan --dataset_mode unaligned_masked --continue_train --epoch_count 82 --niter 0 --niter_decay 100 --lr 0.00002

----------------- Options ---------------
               batch_size: 1                             
                    beta1: 0.5                           
          checkpoints_dir: ./checkpoints                 
           continue_train: True                          	[default: False]
                crop_size: 256                           
                 dataroot: /gdrive/MyDrive/LeafGANDataSet/Healthy2Rust	[default: None]
             dataset_mode: unaligned_masked              	[default: unaligned]
                direction: AtoB                          
              display_env: main                          
             display_freq: 400                           
               display_id: -1                            
            display_ncols: 4                             
             display_port: 8097                          
           display_server: http://localhost              
          display_winsize: 256                           
                    e

**intergrate logs to weights and biases**

In [ ]:
!pip install wandb
!wandb login 52b3b279a9d634741e39bd8541fece124dfb8b88

     |████████████████████████████████| 1.7 MB 4.3 MB/s 
     |████████████████████████████████| 143 kB 43.1 MB/s 
     |████████████████████████████████| 180 kB 55.2 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=aef418d10c62a840bf35d9b5a98148299dba92788a2a2ee4c4bcbb1888c951b4
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
wab = wandb.init(project="LeafGAN", entity="seminar")
lines = []
LOGS_PATH = '/gdrive/MyDrive/LeafGANDataSet/LeafGAN/checkpoints/Healthy2Rust_leafGAN/loss_log.txt'
with open(LOGS_PATH) as f:
    lines = f.readlines()
lines = lines[1:-1]

for l in lines:
  l = l.split(')')[1].split(" ")[1:-1]
  losses = {}
  losses["D_A"] = float(l[1])
  losses["G_A"] = float(l[3])
  losses["cycle_A"] = float(l[5])
  losses["idt_A"] = float(l[7])
  losses["D_B"] = float(l[9])
  losses["G_B"] = float(l[11])
  losses["cycle_B"] = float(l[13])
  losses["idt_B"] = float(l[15])
  wandb.log(losses)
  
wab.finish()

D_A,▂▇█▃▃▂▅▅▆▄▄▅▃▁▂▅▄▄▄▅▄▃▁▂▂▇▅▂▄▄▄█▃▄▁▃▂▂▅▅
D_B,▆█▂▂▃▃▂▃▄▁▃▁▂▆▂▂▅▄▂▅▃▂▃▄▂▆▃▄▂▅▃▄▄▃▄▂▁▇▃▄
G_A,▅▅▂▂▄▇▄▃▄▅▆▄▃▃▃▆▂▃█▃▂▇▅▃▃▂▃▄▅▃▆▃▅▃▇▆▄▁▇▂
G_B,▄▁▄▅▅▄▃▅▃▄▄▃▅█▇▃▇▃▃▂▄▃▆▇▅▅▅▇▂▃▃▃▆▂▅▄▄▄▃▄
cycle_A,█▃▆▃▂▅▃▄▇▅▃▄▃▃▂▂█▅▂▃▁▆▁▂▁▄▂▄▂▄▂▃▁▁▂▁▁▃▄▄
cycle_B,█▃▃▃▃▇▃█▂▂▅▂▂▄▄▃▂▄▂▃▂▃▂▂▂▃▂▂▃▄▁▃▁▂▄▃▃▂▂▂
idt_A,█▃▂▃▂▆▅▆▃▃▃▂▂▃▃▃▂▃▂▂▂▃▃▂▂▂▁▂▂▃▁▃▁▄▃▁▂▂▂▁
idt_B,█▃▇▄▃▆▃▄▄▆▄▄▅▃▃▂▆▆▂▄▂▆▁▃▁▃▂▄▂▆▁▄▁▁▃▃▂▃▄▃
D_A,0.141
D_B,0.103
G_A,0.284


**test**

In [ ]:
!python test.py --dataroot /gdrive/MyDrive/LeafGANDataSet/Healthy2Rust --name Healthy2Rust_leafGAN --model leaf_gan